### For Juan and Walker

In [1]:
# Based on plot_all_spectra_NEW.ipynb
# Also based on modeling_core.ipynb

In [2]:
# For parallelization, which helps to make convolution faster
import dask
dask.config.set(scheduler = 'threads', num_workers = 8)
from dask.diagnostics import ProgressBar
ProgressBar().register()
import warnings
warnings.filterwarnings('ignore')

In [3]:
from spectral_cube import OneDSpectrum
import pyspeckit
from astropy.io import fits
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np

make: *** No targets specified and no makefile found.  Stop.


In [4]:
results = '/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/'

In [5]:
# Get all spws for B3
freq_spws = [
'87_spw25',
'89_spw27',
'91_spw25', 
'93_spw27',
'95_spw25', 
'97_spw27',
'98_spw29',
'99_spw31',
'102_spw23',
'102_spw29',
'103_spw31',
'106_spw29',
'107_spw31',
'110_spw29',
'111_spw31',
'112_spw27',
'114_spw29']

In [6]:
from spectral_cube import SpectralCube
from astropy import coordinates
from astropy import wcs

for spw_name in freq_spws:
    # Get cube based on spw name and smooth it to a common beam
    ind = freq_spws.index(spw_name) # Silly way to do this, but easiest at the moment
    cube = SpectralCube.read(results+'source_ab_'+freq_spws[ind]+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.image', use_dask = True)
    cube_common_beam = cube.beams.common_beam(max_iter = 20, max_epsilon = 0.01)
    smoothed_cube = cube.convolve_to(cube_common_beam) # Convert from VaryingResolution to regular
    # Extract spectrum for a coordinate in central core region
    crd = coordinates.SkyCoord("17:46:10.63 -28:42:17.8", frame='fk5', unit=(u.h, u.deg)) # This is the central core, about
    x, y = map(int, smoothed_cube.wcs.celestial.world_to_pixel(crd))
    spectrum = smoothed_cube[:, y, x].to(u.K)
    spectrum.write(f'B3_spectra_for_juan/{spw_name}.fits', format='fits')

[########################################] | 100% Completed | 93.77 s
[########################################] | 100% Completed | 93.65 s
[########################################] | 100% Completed | 93.40 s
[########################################] | 100% Completed | 92.94 s
[########################################] | 100% Completed | 93.32 s
[########################################] | 100% Completed | 92.86 s
[########################################] | 100% Completed | 93.83 s
[########################################] | 100% Completed | 93.71 s
[########################################] | 100% Completed | 94.13 s
[########################################] | 100% Completed | 94.28 s
[########################################] | 100% Completed | 93.74 s
[########################################] | 100% Completed | 93.90 s
[########################################] | 100% Completed | 94.13 s
[########################################] | 100% Completed | 93.78 s
[###################

In [45]:
from astropy.io import fits
from astropy import wcs
from astropy import units as u
from radio_beam import Beam
from spectral_cube import OneDSpectrum

# Verify that you can read in and plot the spectrum in this form
# Paraphrased from spectral_cube source code

spw_name_example = '106_spw29'

with fits.open(f'{spw_name_example}.fits') as hdul:
    # spectrum_in = OneDSpectrum.from_hdu(hdul) # Doesn't work, gives "beam must be a radio_beam.Beams object;" Adam made bugfix on 2022/12/01
    mywcs = wcs.WCS(hdul[0].header)
    unit = u.Unit(hdul[0].header["BUNIT"])
    beam = Beam.from_fits_header(hdul[0].header)
    spectrum_in = OneDSpectrum(hdul[0].data, unit = unit, wcs = mywcs, header = hdul[0].header, beam = beam)